In [ ]:
"""
Created on Thu Jul 06 11:53 2023

Look at the minimum thickness of ice shelves

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import basal_melt_param.useful_functions as uf
from tqdm.notebook import tqdm

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
map_lim = [-3000000,3000000]


In [ ]:
BedMachine_folder = '_reduced5'


In [ ]:
inputpath_data = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/raw/BEDMACHINE_RTOPO/'
outputpath_mask ='/bettik/burgardc/DATA/BASAL_MELT_PARAM/interim/ANTARCTICA_IS_MASKS/BedMachine_for_comparison'+BedMachine_folder+'/'


In [ ]:
file_mask_orig = xr.open_dataset(inputpath_data+'BedMachineAntarctica_2020-07-15_v02'+BedMachine_folder+'.nc')
file_mask_orig_cut = uf.cut_domain_stereo(file_mask_orig, map_lim, map_lim)

In [ ]:
bedmachine_mask = xr.open_dataset(outputpath_mask + 'BedMachine'+BedMachine_folder+'_isf_masks_and_info_and_distance_new.nc')

In [ ]:
bedmachine_isfmask_oneFRIS = bedmachine_mask['ISF_mask'].where(bedmachine_mask['ISF_mask'] != 21, 11)

In [ ]:
nisf_list = bedmachine_mask.Nisf.drop_sel(Nisf=21)

In [ ]:
mean_list = []
for kisf in tqdm(nisf_list):
    kisf_thickness = file_mask_orig_cut['thickness'].where(bedmachine_isfmask_oneFRIS == kisf)
    mean_kisf_thickness = kisf_thickness.mean()
    mean_list.append(mean_kisf_thickness.assign_coords({'Nisf': kisf}))
mean_da = xr.concat(mean_list, dim='Nisf')

In [ ]:
bedmachine_isfmask_oneFRIS

In [ ]:
kisf_min_ID = mean_da.Nisf.where(mean_da == mean_da.min(), drop=True)
bedmachine_mask['isf_name'].sel(Nisf = kisf_min_ID).load()

In [ ]:
kisf_min_ID = mean_da.Nisf.where(mean_da < mean_da.quantile(0.01), drop=True)
bedmachine_mask['isf_name'].sel(Nisf = 199).load()

In [ ]:
bedmachine_isfmask_oneFRIS.where(bedmachine_isfmask_oneFRIS == 199).plot()

In [ ]:
bedmachine_data['mask'].where(bedmachine_data['mask'] == 3).plot()

In [ ]:
bedmachine_data['mask'].where(bedmachine_data['mask'] == 3).plot()

In [ ]:
bedmachine_data['thickness'].where(bedmachine_data['mask'] == 3).quantile(0.01)

In [ ]:
isf_thickness = bedmachine_data['thickness'].where(bedmachine_data['mask'] == 3)

In [ ]:
isf_thickness.where(isf_thickness < 7.8).plot()

In [ ]:
bedmachine_mask